# Часть 1: Общие задачи - EDA СберАвтоподписка

**Бизнес-контекст:** Сервис "СберАвтоподписка" - долгосрочная аренда авто.

**Целевые действия:**
- sub_car_claim_click
- sub_car_claim_submit_click
- sub_open_dialog_click
- sub_custom_question_submit_click
- sub_call_number_click
- sub_callback_submit_click
- sub_submit_success
- sub_car_request_submit_click

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
print('✅ Библиотеки загружены')

## 1.1 Загрузка данных

In [ ]:
import os

def load_data(filename, path='../data/'):
    filepath_pkl = os.path.join(path, filename + '.pkl')
    filepath_csv = os.path.join(path, filename + '.csv')
    
    if os.path.exists(filepath_pkl):
        print(f'📊 Загружаем {filename}.pkl')
        return pd.read_pickle(filepath_pkl)
    elif os.path.exists(filepath_csv):
        print(f'📊 Загружаем {filename}.csv')
        return pd.read_csv(filepath_csv)
    else:
        raise FileNotFoundError(f'Файл {filename} не найден')

ga_sessions = load_data('ga_sessions')
ga_hits = load_data('ga_hits')

print(f'\n✅ GA Sessions: {ga_sessions.shape}')
print(f'✅ GA Hits: {ga_hits.shape}')

In [ ]:
# Информация о данных
print('GA Sessions колонки:', list(ga_sessions.columns))
print('\nGA Hits колонки:', list(ga_hits.columns))

print('\nПропуски в Sessions:')
print(ga_sessions.isnull().sum())
print('\nПропуски в Hits:')
print(ga_hits.isnull().sum())

## 1.2 Базовая очистка

In [ ]:
# Удаление дубликатов
sessions_clean = ga_sessions.drop_duplicates()
hits_clean = ga_hits.drop_duplicates()

print(f'Sessions: {len(ga_sessions)} → {len(sessions_clean)}')
print(f'Hits: {len(ga_hits)} → {len(hits_clean)}')

# Типизация
sessions_clean['visit_date'] = pd.to_datetime(sessions_clean['visit_date'])
hits_clean['hit_date'] = pd.to_datetime(hits_clean['hit_date'])

cat_cols = ['utm_source', 'utm_medium', 'device_category', 'device_os', 'geo_city']
for col in cat_cols:
    if col in sessions_clean.columns:
        sessions_clean[col] = sessions_clean[col].astype('category')

print('\n✅ Очистка завершена')

## 1.3 Визуализации

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# UTM Source
sessions_clean['utm_source'].value_counts().head(8).plot(kind='barh', ax=axes[0,0])
axes[0,0].set_title('Топ-8 utm_source')

# Device category
sessions_clean['device_category'].value_counts().plot(kind='pie', ax=axes[0,1], autopct='%1.1f%%')
axes[0,1].set_title('Типы устройств')
axes[0,1].set_ylabel('')

# Geo city
sessions_clean['geo_city'].value_counts().head(8).plot(kind='barh', ax=axes[1,0])
axes[1,0].set_title('Топ-8 городов')

# Динамика визитов
sessions_clean.groupby('visit_date').size().plot(ax=axes[1,1])
axes[1,1].set_title('Визиты по дням')

plt.tight_layout()
plt.show()

## 1.4 Целевая переменная

In [ ]:
target_actions = [
    'sub_car_claim_click', 'sub_car_claim_submit_click',
    'sub_open_dialog_click', 'sub_custom_question_submit_click',
    'sub_call_number_click', 'sub_callback_submit_click',
    'sub_submit_success', 'sub_car_request_submit_click'
]

# Находим сессии с целевыми действиями
target_sessions = hits_clean[
    hits_clean['event_action'].isin(target_actions)
]['session_id'].unique()

sessions_clean['target'] = sessions_clean['session_id'].isin(target_sessions).astype(int)

print(f'Целевых действий: {len(target_sessions)}')
print(f'Конверсия: {sessions_clean["target"].mean():.2%}')
print('\nРаспределение:')
print(sessions_clean['target'].value_counts())

## 1.5 Корреляции

In [ ]:
# Числовые признаки
num_cols = sessions_clean.select_dtypes(include=[np.number]).columns

if len(num_cols) > 1:
    plt.figure(figsize=(10, 8))
    sns.heatmap(sessions_clean[num_cols].corr(), annot=True, cmap='coolwarm', center=0)
    plt.title('Корреляционная матрица')
    plt.show()

print('✅ EDA завершен')